# Save and Restore Classifer
### Can be use to predictor or retrain on aditional data

## Create Enviroment

In [1]:
'''Set output directory for model files '''

model_out = 'model_out'
OUTPUT_DIR = model_out



In [2]:
'''Import Session Variables '''

import dill

# all variables for trained BERT classifer as Jupyter Kernal
dill.load_session('product_classifier.db')



W0806 21:32:20.595706  4932 deprecation_wrapper.py:119] From C:\Users\ehshan\AppData\Local\Programs\Python\Python37\lib\site-packages\bert\optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



## Used Save model to predict

In [3]:
'''Define the remote url of sample data '''

product_dir = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/'
sample_file = 'sample_us.tsv'



In [4]:
'''Load data from remote list '''

def load_sample_from_remote(force_download = False):
    
    # load data from url
    sample_dataset = tf.keras.utils.get_file(
        fname = sample_file, 
        origin = '{}{}'.format(product_dir, sample_file), 
        extract = False)

    fields = ['review_body']
    
        # read data to df
    df = pd.read_csv(sample_dataset, sep='\t', header=0, skipinitialspace=True, usecols=fields, encoding='utf-8')
    
    sample = df.values.tolist()

    # fatten
    sample = [item for items in sample for item in items]

    return sample

    

In [5]:
'''Assign sample list'''

review_samples = load_sample_from_remote()



In [6]:
'''Predict class for all sample reviews'''


def predict_class(sentences):
    labels = ['1', '2', '3', '4', '5']
    input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 4) for x in sentences]
    input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
    predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
    predictions = estimator.predict(predict_input_fn)
    return [(sentence, prediction['probabilities'], labels[prediction['labels']]) for sentence, prediction in zip(sentences, predictions)]



sample_predictions = predict_class(review_samples)

sample_predictions


W0806 21:33:05.537209  4932 deprecation_wrapper.py:119] From C:\Users\ehshan\AppData\Local\Programs\Python\Python37\lib\site-packages\bert\run_classifier.py:774: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

W0806 21:33:20.147429  4932 deprecation.py:506] From <ipython-input-11-7029fd60352c>:43: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0806 21:33:20.641594  4932 deprecation.py:323] From C:\Users\ehshan\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\ops\array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0806 21:33:22.958260  4932 depr

[('Excellent!!!',
  array([-6.0958395 , -6.9492207 , -6.1457515 , -4.323141  , -0.01878776],
        dtype=float32),
  '5'),
 ('Great quality wooden track (better than some others we have tried). Perfect match to the various vintages of Thomas track that we already have. There is enough track here to have fun and get creative incorporating your key pieces with track splits, loops and bends.',
  array([-6.961491  , -7.4736543 , -6.7358975 , -4.0409675 , -0.02049203],
        dtype=float32),
  '5'),
 ('Cards are not as big as pictured.',
  array([-3.4434505 , -1.4038181 , -0.37096903, -3.5854554 , -5.381879  ],
        dtype=float32),
  '3'),
 ('my daughter loved it and i liked the price and it came to me rather than shopping with a ton of people around me. Amazon is the Best way to shop!',
  array([-7.186801  , -7.381016  , -5.982897  , -2.8688533 , -0.06258312],
        dtype=float32),
  '5'),
 ("Do not buy these! They break very fast I spun then for 15 minutes and the end flew off don